In [1]:
import mwpf
# import mwpf_rational as mwpf

mwpf.Visualizer.embed()

In [2]:
code = mwpf.CodeCapacityColorCode(d=5, p=0.001, max_weight=1)
code.set_defect_vertices([2,3,7])
initializer = code.get_initializer()
solver = mwpf.SolverSerialJointSingleHair(initializer)

visualizer = mwpf.Visualizer(positions=code.get_positions())

# instead of using `solver.solve`, we can manually load the syndrome and manipulate the dual variables
syndrome = code.get_syndrome()
solver.load_syndrome(syndrome, visualizer)
visualizer.show()

The algorithm works in two phases: the searching phase and tuning phase. They have different model for dual variable updates
- searching phase: set the direction of all dual variables (initially all +1) and update until an Obstacle is detected
- tuning phase: update a subset of dual variables given their direction, remaining others to be the same

Please read our paper about why this two-phase process helps improve the performance.

At the beginning, each defect vertex $v$ corresponds a node $S = (\{v\}, \varnothing)$. You can use the index to get a pointer to each of them.

In [3]:
defect_nodes = [solver.get_node(i) for i in range(4)]
for defect_node in defect_nodes:
    print(repr(defect_node))
assert solver.get_node(1) == solver.get_node(1), "the same node should be equal"

DualNode { index: 0, dual_variable: OrderedFloat(0.0), grow_rate: OrderedFloat(1.0), hair: {5, 6, 9, 10, 12, 13} }
DualNode { index: 1, dual_variable: OrderedFloat(0.0), grow_rate: OrderedFloat(1.0), hair: {2, 3, 6, 7, 10, 11} }
DualNode { index: 2, dual_variable: OrderedFloat(0.0), grow_rate: OrderedFloat(1.0), hair: {10, 11, 13, 14, 15, 16} }
None


Or, if you know the invalid cluster $V_S$ and $E_S$, you can also find the node

In [4]:
node = solver.find_node(vertices={3}, edges={})
print(node)

Node(1)


Initially, all the dual variables have $y_S = 0$. You can inspect the information of each node like below. 

In [5]:
print("index:", node.index)
print("dual_variable:", node.dual_variable)
print("grow_rate:", node.grow_rate)
print("hair:", node.hair)
print("vertices:", node.vertices)
print("edges:", node.edges)

index: 1
dual_variable: 0
grow_rate: 1
hair: {2, 3, 6, 7, 10, 11}
vertices: {3}
edges: set()


We can grow the dual variables by some amount. Suppose the grow rate of each dual variable is $\Delta y_S$, then each dual variable is updated to $y_S' = y_S + \Delta y_S \times l$ where $l$ is the length of growth.

In [6]:
length = mwpf.Rational(1, 3)
print("length:", repr(length))
print(f"{length} = {length.float()}")
print(f"1/3 + 1/7 = {mwpf.Rational(1, 3) + mwpf.Rational(1, 7)}")

length: OrderedFloat(0.3333333333333333)
0.3333333333333333 = 0.3333333333333333
1/3 + 1/7 = 0.47619047619047616


We can then ask, given the current grow rate $\Delta y_S$, what is the maximum length that we can grow? Specifically, we are bounded by the dual constraints $\sum_{S \in \mathcal{O} | e \in \delta(S)} y_S \le w_e, \forall e \in E$. Besides, we have $y_S \ge 0, \forall S \in \mathcal{O}$ non-negativity constriants. We would like to ask, what is the maximum length $l$, such that no dual constraint is violated; and if the maximum length is $0$, then what is the **obstacle**, a dual constraint, that forbids the current solution to grow further.

In [7]:
dual_report = solver.dual_report()
print(dual_report)
match dual_report:
    case mwpf.DualReport.Unbounded():
        print("Unbounded")
    case mwpf.DualReport.ValidGrow(length):
        print(f"ValidGrow({length})")
    case mwpf.DualReport.Obstacles(obstacles):
        print(f"Obstacles({obstacles})")
    case _:
        raise NotImplementedError

ValidGrow(0.3333333333333333)
ValidGrow(0.3333333333333333)


In [8]:
solver.grow(mwpf.Rational(1, 3))

In [9]:
visualizer.snapshot("grow", solver)

In [10]:
visualizer.show()

Given that the center hyperedge is already tight, we cannot grow it anymore. We can programmatically get this information using the `dual_report` function again.

In [11]:
dual_report = solver.dual_report()
print(dual_report)
match dual_report:
    case mwpf.DualReport.Obstacles(obstacles):
        for obstacle in obstacles:
            match obstacle:
                case mwpf.Obstacle.Conflict(edge_index):
                    print(f"Conflict on edge {edge_index}")
                case mwpf.Obstacle.ShrinkToZero(node):
                    print(f"ShrinkToZero on node {node}")
    case _:
        raise NotImplementedError

Obstacles([Conflict { edge_index: 10 }])
Conflict on edge 10


Now that we know there is a Conflict, i.e., dual constraint $\sum_{S \in \mathcal{O} | e \in \delta(S)} y_S = w_e$ (tight constraint) while $\sum_{S \in \mathcal{O} | e \in \delta(S)} \Delta y_S > 0$ (still trying to increase the summation), we need to do something about the grow rate to remove the grow rate.
This is simply: we first query what are the dual nodes on the conflicting edge and then set all of them to 0.

In [12]:
conflict_nodes = solver.get_edge_nodes(edge_index)
for node in conflict_nodes:
    solver.set_grow_rate(node, mwpf.Rational(0))
print(conflict_nodes)

[DualNode { index: 0, dual_variable: OrderedFloat(0.3333333333333333), grow_rate: OrderedFloat(0.0), hair: {5, 6, 9, 10, 12, 13} }, DualNode { index: 1, dual_variable: OrderedFloat(0.3333333333333333), grow_rate: OrderedFloat(0.0), hair: {2, 3, 6, 7, 10, 11} }, DualNode { index: 2, dual_variable: OrderedFloat(0.3333333333333333), grow_rate: OrderedFloat(0.0), hair: {10, 11, 13, 14, 15, 16} }]


In [13]:
dual_report = solver.dual_report()
print(dual_report)
assert type(dual_report) == mwpf.DualReport.Unbounded

Unbounded


Now that the dual phase is unboundede, there is no more Obstacles to be solved. We may encounter two cases:
1. A parity factor is found, then the problem is feasible and we can output the solution and the primal-dual gap.
2. No parity factor is found and it is unbounded, then the problem is infeasible because we can grow the dual variables indefinitely without harming any dual constraint. That means there exists an invalid subgraph $S = (V_S, E_S)$ such that $\delta (S) = \varnothing$. By definition of invalid subgraph, this subgraph does not have any parity factor solution and thus the overall problem is infeasible.

In [14]:
visualizer.snapshot("solved", solver, mwpf.Subgraph([10]))

In [15]:
visualizer.show()